# 10: Number of Automobiles

There are strong theoretical and empirical reasons to expect that a variety of decision maker related variables such as income, car availability, residential location, number of workers in the household and others, influence workers’ choice of travel mode.

Model 10 considers the number of automobiles in the household. (pp. 120)

Import larch and verify that the version number is correct (as of 1/30/2024 should be 6.0.17)

In [ ]:
import larch as lx

In [ ]:
lx.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = lx.examples.MTC(format="dataset")
d

In [ ]:
m = lx.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:`idco` data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:`LinearFunction` objects for each alternative code.

In [ ]:
from larch import PX, P, X

for a in [2, 3]:
    m.utility_co[a] = +P("hhinc#2,3") * X("hhinc") + P(f"numveh#{a}") * X("numveh")

for a in [4, 5, 6]:
    m.utility_co[a] = +P(f"hhinc#{a}") * X("hhinc") + P(f"numveh#{a}") * X("numveh")

Sometimes we may want to define a part of the utility function that is common across all (or almost all)
of the alternatives.  We can access a dictionary (more generically called a "mapping") of alternative
codes to alternative names, which can be found via the `Dataset.dc.alts_mapping` attribute:

In [ ]:
d.dc.alts_mapping

Using this like a standard Python dictionary, we can iterate over all the alternatives, skipping 1, and
setting alternative specific constants (ASC's) for the rest.

In [ ]:
for a, name in d.dc.alts_mapping.items():
    if a == 1:
        continue
    m.utility_co[a] += P("ASC_" + name)

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +PX("totcost")
    + P("motorized_time") * X("(altid <= 4) * tottime")
    + P("nonmotorized_time") * X("(altid > 4) * tottime")
    + P("motorized_ovtbydist") * X("(altid <= 4) * ovtt/dist")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 10, Autos per Household"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
# TEST
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, method="bhhh")

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    ("Ownership", "numveh.*"),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
from pytest import approx

assert result.loglike == approx(-3501.6427674451616)
revealed_x = dict(zip(m.pnames, result.x))
expected_x = {
    "ASC_Bike": -2.220318494507017,
    "ASC_SR2": -2.0539566005386884,
    "ASC_SR3+": -3.643068905518874,
    "ASC_Transit": 0.5738157734892145,
    "ASC_Walk": -0.4403348747865274,
    "hhinc#2,3": -0.001955401039169736,
    "hhinc#4": -0.0013233467107319043,
    "hhinc#5": -0.009513056978703192,
    "hhinc#6": -0.004156633998143166,
    "motorized_ovtbydist": -0.17850538190676227,
    "motorized_time": -0.03782098120360213,
    "nonmotorized_time": -0.047506740267090244,
    "numveh#2": -0.035252140629887845,
    "numveh#3": 0.07226723634772537,
    "numveh#4": -0.5544590047253285,
    "numveh#5": -0.22920682058782335,
    "numveh#6": -0.36557711952920985,
    "totcost": -0.004059016748597727,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )